# Homework 3

Download dataset

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv


--2023-10-02 16:44:05--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M   758KB/s    in 1.9s    

2023-10-02 16:44:07 (758 KB/s) - ‘data.csv’ saved [1475504/1475504]



Data preparation and feature selection

In [7]:
import pandas as pd

data = pd.read_csv("./data.csv")
selected_columns = [
    "Make",
    "Model",
    "Year",
    "Engine HP",
    "Engine Cylinders",
    "Transmission Type",
    "Vehicle Style",
    "highway MPG",
    "city mpg",
    "MSRP"
]
data = data.fillna(value={
    col: 0
    for col in selected_columns
})
data = data[selected_columns]
data.columns = data.columns.str.replace(' ', '_').str.lower()
data = data.rename(columns={"msrp": "price"})
# data = data.fillna(0, )
data

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500
...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920


## Question 1

What is the most frequent observation (mode) for the column `transmission_type`?

- `AUTOMATIC`
- `MANUAL`
- `AUTOMATED_MANUAL`
- `DIRECT_DRIVE`


In [10]:
data.groupby(by='transmission_type').count()

,make,model,year,engine_hp,engine_cylinders,vehicle_style,highway_mpg,city_mpg,price
transmission_type,,,,,,,,,
AUTOMATED_MANUAL,626,626,626,626,626,626,626,626,626
AUTOMATIC,8266,8266,8266,8266,8266,8266,8266,8266,8266
DIRECT_DRIVE,68,68,68,68,68,68,68,68,68
MANUAL,2935,2935,2935,2935,2935,2935,2935,2935,2935
UNKNOWN,19,19,19,19,19,19,19,19,19


In [8]:
data['transmission_type'].mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

The frequent observation of the column transission_type is `automatic`

## Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

- `engine_hp` and `year`
- `engine_hp` and `engine_cylinders`
- `highway_mpg` and `engine_cylinders`
- `highway_mpg` and `city_mpg`


In [11]:
data.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
dtype: object

In [16]:
numerical_features = [
    'year',
    'engine_hp',
    'engine_cylinders',
    'highway_mpg',
    'city_mpg',
    'price',   
]

data[numerical_features].corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [32]:
import numpy as np

correlation_table = data[numerical_features].corr().abs()

# Filter the 1s correlation values
filtered_correlation_table = correlation_table.mask(np.eye(len(correlation_table), dtype=bool))
filtered_correlation_table

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,NaN,0.338714,0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,NaN,0.774851,0.415707,0.424918,0.650095
engine_cylinders,0.040708,0.774851,NaN,0.614541,0.587306,0.526274
highway_mpg,0.258240,0.415707,0.614541,NaN,0.886829,0.160043
city_mpg,0.198171,0.424918,0.587306,0.886829,NaN,0.157676
price,0.227590,0.650095,0.526274,0.160043,0.157676,NaN


In [31]:
print(f"Correlation of engine_hp and year: ", filtered_correlation_table['engine_hp']['year'])
print(f"Correlation of engine_hp and engine_cylinders: ", filtered_correlation_table['engine_hp']['engine_cylinders'])
print(f"Correlation of highway_mpg and engine_cylinders: ", filtered_correlation_table['highway_mpg']['engine_cylinders'])
print(f"Correlation of highway_mpg and city_mpg: ", filtered_correlation_table['highway_mpg']['city_mpg'])

Correlation of engine_hp and year:  0.3387141847624335
Correlation of engine_hp and engine_cylinders:  0.7748509807813221
Correlation of highway_mpg and engine_cylinders:  0.6145414173953352
Correlation of highway_mpg and city_mpg:  0.8868294962591425


The answer is: `highway_mpg and city_mpg`

## Question 3

* Calculate the mutual information score between `above_average` and other categorical variables in our dataset. 
  Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the lowest mutual information score?
  
- `make`
- `model`
- `transmission_type`
- `vehicle_style`

Make price binary

In [265]:
data['above_average'] = data['price'].map(lambda x: 1 if x > data['price'].mean() else 0)
data

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500,0
...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,46120,1
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,56670,1
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50620,1
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,50920,1


Split train/valid/test

In [266]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=42)

In [267]:
len(df_train), len(df_val), len(df_test)


(7624, 1907, 2383)

In [268]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [269]:
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values


df_train_with_target = df_train.copy()
del df_train['above_average']
del df_val['above_average']
del df_test['above_average']


In [270]:
from sklearn.metrics import mutual_info_score

def mutual_info_above_average_score(series):
    return mutual_info_score(series, df_train_with_target.above_average)

categorical_features = [
    'make',
    'model',
    'transmission_type',
    'vehicle_style'
]

mi = df_train_with_target[categorical_features].apply(mutual_info_above_average_score)
mi.sort_values(ascending=False)

model                0.460514
make                 0.240114
vehicle_style        0.084334
transmission_type    0.021262
dtype: float64

In [271]:
print(f"The lowest variable that has a lowest mutual information score is: {mi.sort_values(ascending=False).index[-1]}")

The lowest variable that has a lowest mutual information score is: transmission_type


## Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.60
- 0.72
- 0.84
- 0.95

In [272]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [273]:
train_dict

[{'make': 'Volkswagen',
  'model': 'Beetle',
  'transmission_type': 'MANUAL',
  'vehicle_style': '2dr Hatchback'},
 {'make': 'Chevrolet',
  'model': 'Camaro',
  'transmission_type': 'MANUAL',
  'vehicle_style': 'Convertible'},
 {'make': 'Toyota',
  'model': 'Sienna',
  'transmission_type': 'AUTOMATIC',
  'vehicle_style': 'Passenger Minivan'},
 {'make': 'Hyundai',
  'model': 'Equus',
  'transmission_type': 'AUTOMATIC',
  'vehicle_style': 'Sedan'},
 {'make': 'Aston Martin',
  'model': 'DB7',
  'transmission_type': 'MANUAL',
  'vehicle_style': 'Convertible'},
 {'make': 'Dodge',
  'model': 'Journey',
  'transmission_type': 'AUTOMATIC',
  'vehicle_style': '4dr SUV'},
 {'make': 'Ford',
  'model': 'Explorer Sport Trac',
  'transmission_type': 'AUTOMATIC',
  'vehicle_style': 'Crew Cab Pickup'},
 {'make': 'Nissan',
  'model': 'Versa Note',
  'transmission_type': 'AUTOMATIC',
  'vehicle_style': '4dr Hatchback'},
 {'make': 'Cadillac',
  'model': 'ATS',
  'transmission_type': 'AUTOMATIC',
  'vehic

In [274]:
df_train.shape

(7624, 10)

Train LogisticRegression model

In [275]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [276]:
y_pred = model.predict_proba(X_val)[:, 1]
above_average_decision = (y_pred >= 0.5)


df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = above_average_decision.astype(int)
df_pred['actual'] = y_val.astype(int)
df_pred['correct'] = df_pred.prediction == df_pred.actual
df_pred


,probability,prediction,actual,correct
0,0.000238,0,0,True
1,0.976125,1,1,True
2,0.002348,0,0,True
3,0.226027,0,0,True
4,0.010216,0,0,True
...,...,...,...,...
1902,0.002243,0,0,True
1903,0.012874,0,0,True
1904,0.018724,0,0,True
1905,0.000125,0,0,True


In [277]:
round(df_pred.correct.mean(), 2)


0.93

## Question 5

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `year`
- `engine_hp`
- `transmission_type`
- `city_mpg`

> **Note**: the difference doesn't have to be positive

In [278]:
def train_log_regression_model(
    X_train,
    X_test,
    y_train,
    y_test,
):
    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_test)[:, 1]
    above_average_decision = (y_pred >= 0.5)


    df_pred = pd.DataFrame()
    df_pred['probability'] = y_pred
    df_pred['prediction'] = above_average_decision.astype(int)
    df_pred['actual'] = y_test.astype(int)
    df_pred['correct'] = df_pred.prediction == df_pred.actual
    return df_pred


def get_model_diff_excluding_some_columns(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    y_train,
    y_test,
    excluded_columns=[]
):
    from sklearn.feature_extraction import DictVectorizer

    dv = DictVectorizer(sparse=False)

    train_dict = df_train[[col for col in categorical_features if col not in excluded_columns and col in df_train.columns.to_list()]].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    test_dict = df_test[[col for col in categorical_features if col not in excluded_columns and col in df_test.columns.to_list()]].to_dict(orient='records')
    X_test = dv.transform(test_dict)

    df_train_excluded_col = df_train.drop(columns=excluded_columns)
    df_test_excluded_col = df_test.drop(columns=excluded_columns)
    print(df_train_excluded_col.columns)

    dv2 = DictVectorizer(sparse=False)

    train_dict = df_train_excluded_col[[col for col in categorical_features if col not in excluded_columns and col in df_train.columns.to_list()]].to_dict(orient='records')
    X_train_exc = dv2.fit_transform(train_dict)

    test_dict = df_test_excluded_col[[col for col in categorical_features if col not in excluded_columns and col in df_test.columns.to_list()]].to_dict(orient='records')
    X_test_exc = dv2.transform(test_dict)

    non_excluded_training_perf = train_log_regression_model(X_train, X_test, y_train, y_test)
    excluded_training_perf = train_log_regression_model(X_train_exc, X_test_exc, y_train, y_test)
    print("Non excluded column training performance. Accuracy:", non_excluded_training_perf['correct'].mean())
    print(f"excluded column = {excluded_columns} training performance. Accuracy:", excluded_training_perf['correct'].mean())
    difference = abs(non_excluded_training_perf['correct'].mean() - excluded_training_perf['correct'].mean())
    print("Difference: ", round(difference, 8))

In [279]:
get_model_diff_excluding_some_columns(
    df_train,
    df_test,
    y_train,
    y_test,
    excluded_columns=['year', 'engine_hp', 'transmission_type', 'city_mpg']
)

Index(['make', 'model', 'engine_cylinders', 'vehicle_style', 'highway_mpg',
       'price'],
      dtype='object')
Non excluded column training performance. Accuracy: 0.9357952161141418
excluded column = ['year', 'engine_hp', 'transmission_type', 'city_mpg'] training performance. Accuracy: 0.9357952161141418
Difference:  0.0


There should be a one dominance feature that has a more significance in prediction results. Let's remove that feature first to make the comparison between 4 columns could appears

In [280]:
default_excluded = []
get_model_diff_excluding_some_columns(
    df_train,
    df_val,
    y_train,
    y_val,
    excluded_columns=default_excluded + ['year']
)

Index(['make', 'model', 'engine_hp', 'engine_cylinders', 'transmission_type',
       'vehicle_style', 'highway_mpg', 'city_mpg', 'price'],
      dtype='object')
Non excluded column training performance. Accuracy: 0.9323544834819087
excluded column = ['year'] training performance. Accuracy: 0.9323544834819087
Difference:  0.0


In [281]:
get_model_diff_excluding_some_columns(
    df_train,
    df_val,
    y_train,
    y_val,
    excluded_columns=default_excluded + ['engine_hp']
)

Index(['make', 'model', 'year', 'engine_cylinders', 'transmission_type',
       'vehicle_style', 'highway_mpg', 'city_mpg', 'price'],
      dtype='object')
Non excluded column training performance. Accuracy: 0.9323544834819087
excluded column = ['engine_hp'] training performance. Accuracy: 0.9323544834819087
Difference:  0.0


In [282]:
get_model_diff_excluding_some_columns(
    df_train,
    df_val,
    y_train,
    y_val,
    excluded_columns=default_excluded + ['transmission_type']
)

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'vehicle_style', 'highway_mpg', 'city_mpg', 'price'],
      dtype='object')
Non excluded column training performance. Accuracy: 0.9323544834819087
excluded column = ['transmission_type'] training performance. Accuracy: 0.9323544834819087
Difference:  0.0


In [283]:
get_model_diff_excluding_some_columns(
    df_train,
    df_val,
    y_train,
    y_val,
    excluded_columns=default_excluded + ['city_mpg']
)

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'price'],
      dtype='object')
Non excluded column training performance. Accuracy: 0.9323544834819087
excluded column = ['city_mpg'] training performance. Accuracy: 0.9323544834819087
Difference:  0.0


Summary: No difference through the four option provided. But, based on the mutual information theory, the lowest mutual information feature will contribute less to the prediction results. Therefore, the feature will give less difference even it is removed from training set. The feature is `transmission_type`

## Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column `price`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver `'sag'`. Set the seed to `42`.
* This model also has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`.
* Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

- 0
- 0.01
- 0.1
- 1
- 10

In [251]:
from sklearn.linear_model import LinearRegression, Ridge

data_2 = data.copy()
data_2['price'] = data_2['price'].map(np.log)
data_2

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,10.739327,1
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,10.612754,1
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,10.500949,0
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,10.290449,0
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,10.448715,0
...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.739002,1
11910,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.945000,1
11911,Acura,ZDX,2012,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.832102,1
11912,Acura,ZDX,2013,300.0,6.0,AUTOMATIC,4dr Hatchback,23,16,10.838011,1


In [258]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(data_2, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=42)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values


df_train_with_target = df_train.copy()
del df_train['price']
del df_val['price']
del df_test['price']

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_features].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_features].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [263]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

def evaluate_model(df_train, df_val, y_train, y_val, alpha=0):
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(df_train, y_train)

    y_pred = model.predict(df_val)
    return rmse(y_val, y_pred)

In [264]:
for alpha in [0, 0.01, 0.1, 1, 10]:
    res = evaluate_model(X_train, X_val, y_train, y_val, alpha)
    print(f"Alpha {alpha} rmse is", round(res, 3))

Alpha 0 rmse is 0.258
Alpha 0.01 rmse is 0.257
Alpha 0.1 rmse is 0.26
Alpha 1 rmse is 0.319
Alpha 10 rmse is 0.578


The alpha value that produce lowest RMSE is 0.01